In [1]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to your dataset
train_data_dir = r"D:\Downloads\Dataset\train"

# Specify image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Extract class names from the folder names
class_names = sorted(os.listdir(train_data_dir))

# Create data generator for training data
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False)  # Disable shuffling to maintain the order of images

# Reshape the input data to include a time dimension
X_train_reshaped = []
y_train = []

for i in range(len(train_generator)):
    images, labels = train_generator[i]
    for j in range(len(images)):
        X_train_reshaped.append(images[j])
        y_train.append(labels[j])

X_train_reshaped = np.array(X_train_reshaped)
y_train = np.array(y_train, dtype=np.int32)

# Add a time dimension to the input data
X_train_reshaped = np.expand_dims(X_train_reshaped, axis=1)

# Define the DBLSTM model
model = Sequential()
model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=(1, img_width, img_height, 3)))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(128, activation='relu', return_sequences=False))
model.add(Dense(len(class_names), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    X_train_reshaped,
    y_train,
    batch_size=batch_size,
    epochs=50)

# Save the trained model
model.save('lung_detection_dblstm.h5')


Found 9971 images belonging to 6 classes.



Epoch 1/50


312/312 [==============================] - 181s 574ms/step - loss: 0.9810 - accuracy: 0.6261
Epoch 2/50
312/312 [==============================] - 180s 576ms/step - loss: 0.6123 - accuracy: 0.7743
Epoch 3/50
312/312 [==============================] - 259s 832ms/step - loss: 0.5021 - accuracy: 0.8163
Epoch 4/50
312/312 [==============================] - 268s 859ms/step - loss: 0.4024 - accuracy: 0.8538
Epoch 5/50
312/312 [==============================] - 266s 850ms/step - loss: 0.3190 - accuracy: 0.8839
Epoch 6/50
312/312 [==============================] - 266s 852ms/step - loss: 0.2439 - accuracy: 0.9132
Epoch 7/50
312/312 [==============================] - 266s 853ms/step - loss: 0.1709 - accuracy: 0.9407
Epoch 8/50
312/312 [==============================] - 265s 849ms/step - loss: 0.1315 - accuracy: 0.9529
Epoch 9/50
312/312 [==============================] - 265s 849ms/step - loss: 0.1212 - accuracy: 0.9582
Epoch 10/50
312/

C:\ProgramData\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [2]:
# Set the path to your test dataset
test_data_dir = r"D:\P_126\Dataset_new\test-20230326T155708Z-001\test"

# Create data generator for test data
test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False)  # Disable shuffling to maintain the order of images

# Reshape the input data to include a time dimension
X_test_reshaped = []
y_test = []

for i in range(len(test_generator)):
    images, labels = test_generator[i]
    for j in range(len(images)):
        X_test_reshaped.append(images[j])
        y_test.append(labels[j])

X_test_reshaped = np.array(X_test_reshaped)
y_test = np.array(y_test, dtype=np.int32)

# Add a time dimension to the input data
X_test_reshaped = np.expand_dims(X_test_reshaped, axis=1)

# Load the trained model
model = load_model('lung_detection_dblstm.h5')

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test_reshaped, y_test)
print(f"Test loss: {loss}")
print(f"Test accuracy: {accuracy}")

Found 1966 images belonging to 6 classes.


NameError: name 'load_model' is not defined

In [ ]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Load the saved model
model_path = 'lung_detection_dblstm.h5'
loaded_model = load_model(model_path)

# Define your custom class names
class_names = ['CARDIOMEGALY', 'COVID', 'NORMAL', 'PNEUMONIA', 'PNEUMOTHORAX', 'TUBERCULOSIS']

# Path to the image you want to predict
image_path = r"D:\P_126\Dataset_new\val-20230326T152503Z-001\val\PNEUMONIA"
# Load and preprocess the image
img = image.load_img(image_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add a batch dimension
img_array = np.expand_dims(img_array, axis=1)  # Add a time dimension
img_array /= 255.0  # Rescale pixel values to [0, 1]

# Make a prediction
predictions = loaded_model.predict(img_array)
predicted_class = np.argmax(predictions)

# Get the predicted class name from the custom class names list
predicted_class_name = class_names[predicted_class]

# Display the predicted class
print(f"Predicted class: {predicted_class_name}")

In [2]:
import os
import numpy as np
from tensorflow.keras.models import Sequential, load_model  # Import load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to your dataset
train_data_dir = r"D:\Downloads\Dataset\train"

# Specify image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Extract class names from the folder names
class_names = sorted(os.listdir(train_data_dir))

# Create data generator for training data
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False)  # Disable shuffling to maintain the order of images

# Reshape the input data to include a time dimension
X_train_reshaped = []
y_train = []

for i in range(len(train_generator)):
    images, labels = train_generator[i]
    for j in range(len(images)):
        X_train_reshaped.append(images[j])
        y_train.append(labels[j])

X_train_reshaped = np.array(X_train_reshaped)
y_train = np.array(y_train, dtype=np.int32)

# Add a time dimension to the input data
X_train_reshaped = np.expand_dims(X_train_reshaped, axis=1)

# Define the DBLSTM model
model = Sequential()
model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=(1, img_width, img_height, 3)))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(128, activation='relu', return_sequences=False))
model.add(Dense(len(class_names), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    X_train_reshaped,
    y_train,
    batch_size=batch_size,
    epochs=30,
    steps_per_epoch=312)

# Save the trained model
model.save('lung_detection_dblstm.h5')

# Set the path to your test dataset
test_data_dir = r"D:\Downloads\Dataset\test"

# Create data generator for test data
test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False)  # Disable shuffling to maintain the order of images

# Reshape the input data to include a time dimension
X_test_reshaped = []
y_test = []

for i in range(len(test_generator)):
    images, labels = test_generator[i]
    for j in range(len(images)):
        X_test_reshaped.append(images[j])
        y_test.append(labels[j])

X_test_reshaped = np.array(X_test_reshaped)
y_test = np.array(y_test, dtype=np.int32)

# Add a time dimension to the input data
X_test_reshaped = np.expand_dims(X_test_reshaped, axis=1)

# Load the trained model
model = load_model('lung_detection_dblstm.h5')

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test_reshaped, y_test)
print(f"Test loss: {loss}")
print(f"Test accuracy: {accuracy}")


Found 16050 images belonging to 6 classes.
Epoch 1/30
312/312 [==============================] - 294s 909ms/step - loss: 1.2366 - accuracy: 0.5152
Epoch 2/30
312/312 [==============================] - 298s 956ms/step - loss: 0.7870 - accuracy: 0.7091
Epoch 3/30
312/312 [==============================] - 297s 951ms/step - loss: 0.6299 - accuracy: 0.7702
Epoch 4/30
312/312 [==============================] - 292s 935ms/step - loss: 0.4444 - accuracy: 0.8373
Epoch 5/30
312/312 [==============================] - 295s 945ms/step - loss: 0.3712 - accuracy: 0.8650
Epoch 6/30
312/312 [==============================] - 296s 948ms/step - loss: 0.2497 - accuracy: 0.9101
Epoch 7/30
312/312 [==============================] - 296s 948ms/step - loss: 0.1822 - accuracy: 0.9344
Epoch 8/30
312/312 [==============================] - 298s 954ms/step - loss: 0.1478 - accuracy: 0.9479
Epoch 9/30
312/312 [==============================] - 295s 945ms/step - loss: 0.0807 - accuracy: 0.9708
Epoch 10/30
312/312 [

C:\Users\Hi\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Found 4110 images belonging to 6 classes.
129/129 [==============================] - 23s 164ms/step - loss: 2.0737 - accuracy: 0.7557
Test loss: 2.0737481117248535
Test accuracy: 0.7557177543640137


In [3]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Load the saved model
model_path = 'lung_detection_dblstm.h5'
loaded_model = load_model(model_path)

# Define your custom class names
class_names = ['CARDIOMEGALY', 'COVID', 'NORMAL', 'PNEUMONIA', 'PNEUMOTHORAX', 'TUBERCULOSIS']

# Path to the image you want to predict
image_path = r"C:\Users\Administrator\Desktop\00000403_001.png"
# Load and preprocess the image
img = image.load_img(image_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add a batch dimension
img_array = np.expand_dims(img_array, axis=1)  # Add a time dimension
img_array /= 255.0  # Rescale pixel values to [0, 1]

# Make a prediction
predictions = loaded_model.predict(img_array)
predicted_class = np.argmax(predictions)

# Get the predicted class name from the custom class names list
predicted_class_name = class_names[predicted_class]

# Display the predicted class
print(f"Predicted class: {predicted_class_name}")


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Administrator\\Desktop\\00000403_001.png'

In [4]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix

# Make predictions on the test data
predictions = model.predict(X_test_reshaped)

# Convert predicted probabilities into class labels
predicted_labels = np.argmax(predictions, axis=1)

# Calculate precision, recall, and confusion matrix
precision = precision_score(y_test, predicted_labels, average=None)
recall = recall_score(y_test, predicted_labels, average=None)
confusion_mat = confusion_matrix(y_test, predicted_labels)

# Print precision, recall, and confusion matrix
print("Precision:", precision)
print("Recall:", recall)
print("Confusion Matrix:")
print(confusion_mat)


129/129 [==============================] - 34s 248ms/step
Precision: [0.85769981 0.73509934 0.82218597 0.91324201 0.57669441 0.71768707]
Recall: [0.64233577 0.64817518 0.73576642 0.87591241 0.7080292  0.92408759]
Confusion Matrix:
[[440  69   3   7 115  51]
 [ 23 444  17   7  95  99]
 [ 16  11 504  32 104  18]
 [  4  12  39 600  18  12]
 [ 19  52  49  11 485  69]
 [ 11  16   1   0  24 633]]


In [1]:
import os
import numpy as np
from tensorflow.keras.models import Sequential, load_model  # Import load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to your dataset
train_data_dir = r"D:\Downloads\Dataset\train"

# Specify image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Extract class names from the folder names
class_names = sorted(os.listdir(train_data_dir))

# Create data generator for training data
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False)  # Disable shuffling to maintain the order of images

# Reshape the input data to include a time dimension
X_train_reshaped = []
y_train = []

for i in range(len(train_generator)):
    images, labels = train_generator[i]
    for j in range(len(images)):
        X_train_reshaped.append(images[j])
        y_train.append(labels[j])

X_train_reshaped = np.array(X_train_reshaped)
y_train = np.array(y_train, dtype=np.int32)

# Add a time dimension to the input data
X_train_reshaped = np.expand_dims(X_train_reshaped, axis=1)

# Define the DBLSTM model
model = Sequential()
model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=(1, img_width, img_height, 3)))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))iaa
model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(128, activation='relu', return_sequences=False))
model.add(Dense(len(class_names), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    X_train_reshaped,
    y_train,
    batch_size=batch_size,
    epochs=20,
    steps_per_epoch=312)

# Save the trained model
model.save('lung_detection_dblstm.h5')

# Set the path to your test dataset
test_data_dir = r"D:\Downloads\Dataset\test"

# Create data generator for test data
test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False)  # Disable shuffling to maintain the order of images

# Reshape the input data to include a time dimension
X_test_reshaped = []
y_test = []

for i in range(len(test_generator)):
    images, labels = test_generator[i]
    for j in range(len(images)):
        X_test_reshaped.append(images[j])
        y_test.append(labels[j])

X_test_reshaped = np.array(X_test_reshaped)
y_test = np.array(y_test, dtype=np.int32)

# Add a time dimension to the input data
X_test_reshaped = np.expand_dims(X_test_reshaped, axis=1)

# Load the trained model
model = load_model('lung_detection_dblstm.h5')

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test_reshaped, y_test)
print(f"Test loss: {loss}")
print(f"Test accuracy: {accuracy}")


Found 16050 images belonging to 6 classes.
Epoch 1/20
312/312 [==============================] - 308s 972ms/step - loss: 1.2724 - accuracy: 0.4934
Epoch 2/20
312/312 [==============================] - 301s 966ms/step - loss: 0.8303 - accuracy: 0.6926
Epoch 3/20
312/312 [==============================] - 298s 957ms/step - loss: 0.6843 - accuracy: 0.7466
Epoch 4/20
312/312 [==============================] - 302s 968ms/step - loss: 0.5411 - accuracy: 0.7973
Epoch 5/20
312/312 [==============================] - 300s 962ms/step - loss: 0.4795 - accuracy: 0.8208
Epoch 6/20
312/312 [==============================] - 308s 989ms/step - loss: 0.3718 - accuracy: 0.8666
Epoch 7/20
312/312 [==============================] - 339s 1s/step - loss: 0.3136 - accuracy: 0.8886
Epoch 8/20
312/312 [==============================] - 299s 960ms/step - loss: 0.2591 - accuracy: 0.9051
Epoch 9/20
312/312 [==============================] - 344s 1s/step - loss: 0.1721 - accuracy: 0.9373
Epoch 10/20
312/312 [======

C:\Users\Hi\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Found 4110 images belonging to 6 classes.
129/129 [==============================] - 26s 188ms/step - loss: 1.7583 - accuracy: 0.7453
Test loss: 1.7583073377609253
Test accuracy: 0.7452554702758789


In [2]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix

# Make predictions on the test data
predictions = model.predict(X_test_reshaped)

# Convert predicted probabilities into class labels
predicted_labels = np.argmax(predictions, axis=1)

# Calculate precision, recall, and confusion matrix
precision = precision_score(y_test, predicted_labels, average=None)
recall = recall_score(y_test, predicted_labels, average=None)
confusion_mat = confusion_matrix(y_test, predicted_labels)

# Print precision, recall, and confusion matrix
print("Precision:", precision)
print("Recall:", recall)
print("Confusion Matrix:")
print(confusion_mat)


129/129 [==============================] - 27s 176ms/step
Precision: [0.78640777 0.798      0.75817923 0.92857143 0.54205607 0.74221669]
Recall: [0.70948905 0.58248175 0.77810219 0.8540146  0.67737226 0.87007299]
Confusion Matrix:
[[486  36   2   4 123  34]
 [ 42 399  31   5 113  95]
 [ 13  10 533  29  94   6]
 [  8   7  53 585  17  15]
 [ 44  36  78   6 464  57]
 [ 25  12   6   1  45 596]]
